In [201]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [202]:
%load_ext autoreload
%autoreload
exec(open("plot_utils.py").read())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [203]:
country_name = 'Italy'
file_name = '../output/2020-05-12-fixed-deltat0-old-normal-100k/inference/{}.npz'.format(country_name)
data = np.load(file_name)



In [214]:
var_names = list(data['var_names'])
incidences = data['incidences']
chain = data['chain']
nsteps, ndim, N, Td1, Td2, τ_model = data['params']
nburn = int(nsteps * 0.6)
sample = chain[:, nburn:, :].reshape(-1, ndim)

medians = [np.median(sample[:,i]) for i in range(len(var_names))]
means = [sample[:,i].mean() for i in range(len(var_names))]
lnprobability = data['lnprobability'][:, nburn:]
MAP = sample[lnprobability.argmax()]

In [215]:
list(zip(var_names, means, medians))

[('Z', 3.9601050233814847, 4.108041306058087),
 ('D', 2.952858246125216, 2.8142292613127284),
 ('μ', 0.5444691321172085, 0.5084318001660746),
 ('β', 1.0128890368125536, 0.9663583420018509),
 ('α1', 0.5609353543575976, 0.5478036100368604),
 ('λ', 0.46764315670323686, 0.4616149381171775),
 ('α2', 0.5631532111592872, 0.5518249697912646),
 ('E0', 1074.3977520583933, 971.0537895234131),
 ('Iu0', 1773.6920256031842, 1872.6298914212605),
 ('Δt0', 1.8295168919974756, 1.630144446797973),
 ('τ', 28.49547104102159, 28.493126774162747)]

## medians

In [216]:
seed_max = 3000
params_bounds = {
    'Z' : (2, 5),
    'D' : (2, 5),
    'μ' : (0.2, 1),
    'β' : (0.8, 1.5),
    'α1' : (0.02, 1),
    'λ'  : (0, 1),
    'α2' : (0.02, 1),
    'E0' : (0, seed_max),
    'Iu0' : (0, seed_max),
    'Δt0' : (1,5) #how much zeros before the first incident
}

## loglikelihood code:
θ = medians
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [217]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

loglik_to_print = np.array([0]*Δt0+list(loglik))
n_padded = np.array(list(n)+[-666])
print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik_to_print,X, Xsum, Y, n_padded)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	0	0	0	0	169
1	-2	14	14	169	388
2	-6	62	76	402	612
3	-4	53	129	688	894
4	-3	97	226	1023	1184
5	-9	93	319	1410	1535
6	-34	78	397	1854	1964
7	-5	250	647	2361	2294
8	-4	238	885	2941	2717
9	-10	240	1125	3602	3232
10	-52	561	1686	4357	3532
11	-105	146	1832	5218	4369
12	-34	667	2499	6201	4823
13	-6	587	3086	7322	5516
14	-22	769	3855	8602	6207
15	-9	778	4633	10062	7094
16	-117	1247	5880	11727	7747
17	-193	1492	7372	13627	8424
18	-315	1797	9169	15796	9100
19	-4	977	10146	18269	10945
20	-394	2313	12459	21091	11851
21	-525	2651	15110	24310	12873
22	-357	2547	17657	27983	14515
23	-1266	90	17747	32172	14425
24	-2111	6230	23977	32172	12567
25	-687	4000	27977	36544	12753
26	-389	3526	31503	40730	13305
27	-709	4207	35710	44808	13110
28	-1605	5322	41032	48820	11759
29	-2662	5986	47018	52791	9714
30	-4238	6557	53575	56732	7075
31	-4246	5560	59135	60650	5416
32	-4110	4789	63924	64551	4511
33	-5707	5249	69173	68435	3132
34	-7302	5210	74383	72305	1777
35	-12812	6153	80536	76160	0
36

## means

In [218]:
## loglikelihood code:
θ = means
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [219]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

loglik_to_print = np.array([0]*Δt0+list(loglik))
n_padded = np.array(list(n)+[-666])
print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik_to_print,X, Xsum, Y, n_padded)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	0	0	0	0	196
1	-3	14	14	196	454
2	-4	62	76	468	730
3	-8	53	129	806	1082
4	-5	97	226	1211	1464
5	-20	93	319	1690	1935
6	-61	78	397	2254	2519
7	-5	250	647	2916	3045
8	-20	238	885	3692	3717
9	-46	240	1125	4602	4543
10	-7	561	1686	5668	5232
11	-237	146	1832	6918	6551
12	-6	667	2499	8383	7601
13	-48	587	3086	10100	9026
14	-33	769	3855	12112	10615
15	-82	778	4633	14470	12601
16	-13	1247	5880	17234	14592
17	-9	1492	7372	20472	16894
18	-6	1797	9169	24266	19543
19	-418	977	10146	28712	23776
20	-26	2313	12459	33922	27566
21	-33	2651	15110	40025	32066
22	-165	2547	17657	47176	37895
23	-3777	90	17747	55552	37805
24	-5	6230	23977	55552	40451
25	-658	4000	27977	64428	45079
26	-1324	3526	31503	73056	50107
27	-1264	4207	35710	81610	54477
28	-919	5322	41032	90187	57810
29	-805	5986	47018	98842	60586
30	-714	6557	53575	107604	62915
31	-1403	5560	59135	116490	66375
32	-2269	4789	63924	125510	70748
33	-2299	5249	69173	134672	74806
34	-2716	5210	74383	143979	79053
35	-2342	6153	80536